In [1]:
import requests
import json
import pandas as pd
import folium
import warnings
(warnings.filterwarnings('ignore'))


##the function to help user find the perfect recipe they want
def recipefinder(food, calories, amount, health_options):
    
    if health_options == "no":
        url = "https://api.edamam.com/search?q="+food+"&app_id=22fe6955&app_key=a8701478a52245bcee06a8c8333289a6&from=0&to="+amount+"&calories="+calories
        response = requests.get(url)
        recipes = response.json()
        return recipes
    
    else:
        url ="https://api.edamam.com/search?q="+food+"&app_id=22fe6955&app_key=a8701478a52245bcee06a8c8333289a6&from=0&to="+amount+"&calories="+calories+"&excluded="+health_options
        response = requests.get(url)
        recipes = response.json()
        return recipes 

    

In [2]:
while True:
    user_input_food = input("Please enter a ingredient you want to cook: ")
    user_input_calories = input("Please enter your desired calorie limit ex.(500): ")
    user_input_allergy = input("Are you allergic to anything? If yes, enter the thing you are allergic of. If no, just enter \"no\": ")
    user_input_amount = input("How many options do you want to see? ")
    results = recipefinder(user_input_food, user_input_calories, user_input_amount, user_input_allergy)
    list_recipes = []
    count = 1
    
    print("Results: ")
    for hit in results['hits']:
        recipe_name = hit['recipe']['label']
        list_recipes.append(recipe_name)
        print(count, recipe_name)
        count += 1
    
    try:
        recipe_choice = input("Indicate which recipe (number) you would like more details on. If you want to restart your search, enter \"r\": ")
        if recipe_choice == 'r':
            continue
        else:
            user_choice = input("Do you want to see the nutrition facts of this recipe? ")
            if user_choice == "Yes":
                
                nutrition_fat = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['FAT'])
                nutrition_sugar = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['SUGAR'])
                nutrition_protein = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['PROCNT'])
                nutrition_ca = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['CA'])
                nutrition_fe = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['FE'])
                nutrition_mg = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['MG'])
                nutrition_zn = pd.Series(data = results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']['ZN'])

                nutrition_dict = {'1': nutrition_fat, '2': nutrition_sugar, '3': nutrition_protein, 
                                  '4': nutrition_ca, '5': nutrition_fe, '6': nutrition_zn, '7': nutrition_mg}
                nutrition_df = pd.DataFrame(nutrition_dict)
                nutrition_df = nutrition_df[['1','2','3','4','5','6','7']]
                nutrition_df                
            else:
                print("FOR NOW")
            
    except ValueError:
        print("Please enter an integer or 'r'. ")
        
    break

Please enter a ingredient you want to cook: beef
Please enter your desired calorie limit ex.(500): 1000
Are you allergic to anything? If yes, enter the thing you are allergic of. If no, just enter "no": no
How many options do you want to see? 5
Results: 
1 Roast Beef
2 Garlic Beef
3 Beef Brisket
4 Beef Kreplach
5 Beef Tacos
Indicate which recipe (number) you would like more details on. If you want to restart your search, enter "r": 3
Do you want to see the nutrition facts of this recipe? Yes


In [14]:
nutrition_df

,1,2,3,4,5,6,7
label,Fat,Sugars,Protein,Calcium,Iron,Zinc,Magnesium
quantity,56.3333,3.335,115.102,178.354,6.44037,3.11941,170.21
unit,g,g,g,mg,mg,mg,mg


In [2]:
user_location = input("Where are you right now? ")

params = {}
params["term"] = "grocery store"
params["location"] = user_location

token = "QpgOEI29NHp0IosYkVAneBhWrIKJBDpnfKmu1xEDua5Y1exka_FLmXcxzj9Ir7IcBtKMfqAY-QxYujYcTpRvmbeet0hzrU-KPAKD_BbrHiCdjKVVcWoD6R0D0S3vWnYx"
headers = {'Authorization': 'Bearer %s' % token}

url = "https://api.yelp.com/v3/businesses/search"
request = requests.get(url, params=params, headers=headers)

data = request.json()

center = (data['businesses'][0]['coordinates']['latitude'],data['businesses'][0]['coordinates']['longitude'])

map = folium.Map(location = center, zoom_start = 10)

grocery_stores = data['businesses']

for grocery_store in grocery_stores:
    lat = grocery_store['coordinates']['latitude']
    long = grocery_store['coordinates']['longitude']
    pos = (lat, long)
    annotation = "%s, %s" % (grocery_store['name'], grocery_store['url'])
    marker = folium.Marker(location=pos, popup=annotation)
    map.add_child(marker)
map

Where are you right now? Syracuse, NY
